## backward selection using sklearn_feature selection

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Define functions

def forwardFeature(indep_X, dep_Y, n):
    forward_list = []
    models = [
        LogisticRegression(solver='lbfgs'),
        RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0),
        DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    ]
    
    # Perform Forward Feature Selection for each model
    for model in models:
        sfs = SequentialFeatureSelector(model, n_features_to_select=n, direction='backward')
        sfs.fit(indep_X, dep_Y)
        forward_features = sfs.transform(indep_X)
        forward_list.append(forward_features)
    
    return forward_list

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0, max_iter=200)  # Increased max_iter
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Navie(X_train, y_train, X_test, y_test):   
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)         

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)
    
def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def forward_classification(acclog, accknn, accnav, accdes, accrf): 
    rfedataframe = pd.DataFrame(index=['Logistic', 'Random', 'DecisionTree'], columns=['Logistic', 'KNN', 'Navie', 'Decision', 'Random'])

    for number, index in enumerate(rfedataframe.index):
        rfedataframe['Logistic'][index] = acclog[number]       
        rfedataframe['KNN'][index] = accknn[number]
        rfedataframe['Navie'][index] = accnav[number]
        rfedataframe['Decision'][index] = accdes[number]
        rfedataframe['Random'][index] = accrf[number]
        
    return rfedataframe

# Load dataset
dataset1 = pd.read_csv("CKD.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True, dtype=int)

indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

# Apply Forward Feature Selection
forward_list = forwardFeature(indep_X, dep_Y, 6) 

# Initialize accuracy lists
acclog = []
accknn = []
accnav = []
accdes = []
accrf = []

# Loop through Forward selected features
for i in forward_list:   
    X_train, X_test, y_train, y_test = split_scalar(i, dep_Y)
    
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test, y_test)
    acclog.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test, y_test)  
    accknn.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test, y_test)  
    accnav.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test, y_test)  
    accdes.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test, y_test)  
    accrf.append(Accuracy)
    
# Compile results
result = forward_classification(acclog, accknn, accnav, accdes, accrf)
print(result)


C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

             Logistic   KNN Navie Decision Random
Logistic         0.99  0.99  0.94     0.97   0.98
Random           0.99  0.96  0.86     0.98   0.99
DecisionTree     0.98  0.98  0.96     0.98   0.98


C:\Users\91934\AppData\Local\Temp\ipykernel_19748\1495370482.py:82: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfedataframe['Logistic'][index] = acclog[number]
C:\Users\91934\AppData\Local\Temp\ipykernel_19748\1495370482.py:83: FutureWarn

## forward selection using sklearn_feature_selection

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Define functions

def forwardFeature(indep_X, dep_Y, n):
    forward_list = []
    models = [
        LogisticRegression(solver='lbfgs'),
        RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0),
        DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    ]
    
    # Perform Forward Feature Selection for each model
    for model in models:
        sfs = SequentialFeatureSelector(model, n_features_to_select=n, direction='forward')
        sfs.fit(indep_X, dep_Y)
        forward_features = sfs.transform(indep_X)
        forward_list.append(forward_features)
    
    return forward_list

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0, max_iter=200)  # Increased max_iter
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Navie(X_train, y_train, X_test, y_test):   
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)         

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)
    
def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def forward_classification(acclog, accknn, accnav, accdes, accrf): 
    rfedataframe = pd.DataFrame(index=['Logistic', 'Random', 'DecisionTree'], columns=['Logistic', 'KNN', 'Navie', 'Decision', 'Random'])

    for number, index in enumerate(rfedataframe.index):
        rfedataframe['Logistic'][index] = acclog[number]       
        rfedataframe['KNN'][index] = accknn[number]
        rfedataframe['Navie'][index] = accnav[number]
        rfedataframe['Decision'][index] = accdes[number]
        rfedataframe['Random'][index] = accrf[number]
        
    return rfedataframe

# Load dataset
dataset1 = pd.read_csv("CKD.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True, dtype=int)

indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

# Apply Forward Feature Selection
forward_list = forwardFeature(indep_X, dep_Y, 6) 

# Initialize accuracy lists
acclog = []
accknn = []
accnav = []
accdes = []
accrf = []

# Loop through Forward selected features
for i in forward_list:   
    X_train, X_test, y_train, y_test = split_scalar(i, dep_Y)
    
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test, y_test)
    acclog.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test, y_test)  
    accknn.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test, y_test)  
    accnav.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test, y_test)  
    accdes.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test, y_test)  
    accrf.append(Accuracy)
    
# Compile results
result = forward_classification(acclog, accknn, accnav, accdes, accrf)
print(result)


C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

             Logistic   KNN Navie Decision Random
Logistic          1.0   1.0  0.87      1.0    1.0
Random           0.97  0.98  0.96     0.99   0.99
DecisionTree     0.98  0.99  0.91     0.99   0.97


C:\Users\91934\AppData\Local\Temp\ipykernel_19748\1955594836.py:82: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfedataframe['Logistic'][index] = acclog[number]
C:\Users\91934\AppData\Local\Temp\ipykernel_19748\1955594836.py:83: FutureWarn

## forward selection using mlxtend_feature_selection

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Define functions

def forwardFeature(indep_X, dep_Y, n):
    forward_list = []
    models = [
        LogisticRegression(solver='lbfgs'),
        RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0),
        DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    ]
    
    # Perform Forward Feature Selection for each model
    for model in models:
        sfs = SequentialFeatureSelector(model,k_features=10, forward=True, floating=False, verbose=0, scoring=None, cv=5, n_jobs=1, pre_dispatch='2n_jobs', clone_estimator=True, fixed_features=None, feature_groups=None)
        sfs.fit(indep_X, dep_Y)
        forward_features = sfs.transform(indep_X)
        forward_list.append(forward_features)
    
    return forward_list

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0, max_iter=200)  # Increased max_iter
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Navie(X_train, y_train, X_test, y_test):   
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)         

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)
    
def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def forward_classification(acclog, accknn, accnav, accdes, accrf): 
    rfedataframe = pd.DataFrame(index=['Logistic', 'Random', 'DecisionTree'], columns=['Logistic', 'KNN', 'Navie', 'Decision', 'Random'])

    for number, index in enumerate(rfedataframe.index):
        rfedataframe['Logistic'][index] = acclog[number]       
        rfedataframe['KNN'][index] = accknn[number]
        rfedataframe['Navie'][index] = accnav[number]
        rfedataframe['Decision'][index] = accdes[number]
        rfedataframe['Random'][index] = accrf[number]
        
    return rfedataframe

# Load dataset
dataset1 = pd.read_csv("CKD.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True, dtype=int)

indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

# Apply Forward Feature Selection
forward_list = forwardFeature(indep_X, dep_Y, 6) 

# Initialize accuracy lists
acclog = []
accknn = []
accnav = []
accdes = []
accrf = []

# Loop through Forward selected features
for i in forward_list:   
    X_train, X_test, y_train, y_test = split_scalar(i, dep_Y)
    
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test, y_test)
    acclog.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test, y_test)  
    accknn.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test, y_test)  
    accnav.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test, y_test)  
    accdes.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test, y_test)  
    accrf.append(Accuracy)
    
# Compile results
result = forward_classification(acclog, accknn, accnav, accdes, accrf)
print(result)


C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

             Logistic   KNN Navie Decision Random
Logistic         0.99  0.99  0.91     0.99   0.99
Random           0.97  0.92  0.96     0.99   0.99
DecisionTree     0.96  0.96  0.96      1.0    1.0


C:\Users\91934\AppData\Local\Temp\ipykernel_19748\3339955794.py:82: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfedataframe['Logistic'][index] = acclog[number]
C:\Users\91934\AppData\Local\Temp\ipykernel_19748\3339955794.py:83: FutureWarn

In [8]:
#!pip install mlxtend

## Backward selection using mlxtend_feature_selection

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Define functions

def forwardFeature(indep_X, dep_Y, n):
    forward_list = []
    models = [
        LogisticRegression(solver='lbfgs'),
        RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0),
        DecisionTreeClassifier(criterion='gini', max_features='sqrt', splitter='best', random_state=0)
    ]
    
    # Perform Forward Feature Selection for each model
    for model in models:
        sfs = SequentialFeatureSelector(model,k_features=10, forward=False, floating=False, verbose=0, scoring=None, cv=5, n_jobs=1, pre_dispatch='2n_jobs', clone_estimator=True, fixed_features=None, feature_groups=None)
        sfs.fit(indep_X, dep_Y)
        forward_features = sfs.transform(indep_X)
        forward_list.append(forward_features)
    
    return forward_list

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0, max_iter=200)  # Increased max_iter
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Navie(X_train, y_train, X_test, y_test):   
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)         

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)
    
def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def forward_classification(acclog, accknn, accnav, accdes, accrf): 
    rfedataframe = pd.DataFrame(index=['Logistic', 'Random', 'DecisionTree'], columns=['Logistic', 'KNN', 'Navie', 'Decision', 'Random'])

    for number, index in enumerate(rfedataframe.index):
        rfedataframe['Logistic'][index] = acclog[number]       
        rfedataframe['KNN'][index] = accknn[number]
        rfedataframe['Navie'][index] = accnav[number]
        rfedataframe['Decision'][index] = accdes[number]
        rfedataframe['Random'][index] = accrf[number]
        
    return rfedataframe

# Load dataset
dataset1 = pd.read_csv("CKD.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True, dtype=int)

indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

# Apply Forward Feature Selection
forward_list = forwardFeature(indep_X, dep_Y, 6) 

# Initialize accuracy lists
acclog = []
accknn = []
accnav = []
accdes = []
accrf = []

# Loop through Forward selected features
for i in forward_list:   
    X_train, X_test, y_train, y_test = split_scalar(i, dep_Y)
    
    classifier, Accuracy, report, X_test, y_test, cm = logistic(X_train, y_train, X_test, y_test)
    acclog.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = knn(X_train, y_train, X_test, y_test)  
    accknn.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Navie(X_train, y_train, X_test, y_test)  
    accnav.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = Decision(X_train, y_train, X_test, y_test)  
    accdes.append(Accuracy)
    
    classifier, Accuracy, report, X_test, y_test, cm = random(X_train, y_train, X_test, y_test)  
    accrf.append(Accuracy)
    
# Compile results
result = forward_classification(acclog, accknn, accnav, accdes, accrf)
print(result)


C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

             Logistic   KNN Navie Decision Random
Logistic          1.0   1.0  0.99     0.98    1.0
Random           0.94  0.95  0.76     0.97   0.96
DecisionTree     0.96  0.95  0.96     0.97   0.98


C:\Users\91934\AppData\Local\Temp\ipykernel_19748\1848184140.py:82: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rfedataframe['Logistic'][index] = acclog[number]
C:\Users\91934\AppData\Local\Temp\ipykernel_19748\1848184140.py:83: FutureWarn